<a href="https://colab.research.google.com/github/juanlazarde/financial_scanner/blob/master/unpack_harmonize_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Léete esto primero
Escrito para mi pana Ale y puesto en Notebook, bello para que no se queje. Léete la vaina.

# Latest unpacking of fundamentals code.

**Motivation**: Create dictionary that will replace and harmonize column labels for fundamentals tables, and deduplicate data from same source. We will run this script for every new source of data, i.e. Yahoo, TDA, Alpaca, etc. It is also a pretty good code to re-use for Google Colab.

**Expected Outcome**: CSV/Spreadhseet file with original field names, and equivalent.

**Consideration**: The final mega table with all data from all sources will be filtered with these documents; one per source. If the name does not have an equivalent in this file, it means that it will nor be registered in the database. This way, we will only keep the data from the sources we want.

Google Colab handles Google Spread, otherwise use CSV.

## Instructions
* Code works in Colab, not tested in Jupyter. Won't work on IDE.
* Code is not for production, so not PEP8.
* No atomized functions for easier read.
* Edit the settings below.
* Two things will happen after loading the data, 1) it will stop and show a
  table if there's duplicated data, 2) it will output a dataframe with the
  modifed labels. But, we're interested in the dictionary and the deduplication.

### Standard for names
* all lower caps
* snake_case (perhaps we can parse by '_' and Capitalize next letter for user readability)
* No $ymbols

In [ ]:
# settings
google_colab = False  # using Google Colab tools
get_data = True  # download fresh data
export_df = True  # export the freshly downloaded data.columns
to_csv = True # save it to a csv or if False to Google Sheet
import_df = True  # import the data.columns modified by user
import_csv = True  # get it from csv or if False from Google Sheet
apply_to_df = True  # apply new labels to freshly downloaded data
#file = "/content/drive/MyDrive/columns_in_yahoo"
file = "columns_in_yahoo"

Find your `file` here:

In [ ]:
# current directory
%pwd

In [ ]:
# directories and files here
%ls

In [ ]:
# change directory
# %cd "c:/"

Loading some modules

In [ ]:
# load modules
try:
  import yahooquery as yq
except:
  %pip install yahooquery
  import yahooquery as yq
import pandas as pd
import sys

Custom code below show() and stop(), I've found very useful for Notebook breakpoint and debugging. It shows ALL DataFrame rows and columns. Stop uses ```raise SystemExit``` which is the same as ```sys.exit(0)```, they both generate a SystemExit exception that looks like an error, it is not. Usage:
```
# stop(df)

# stop(True, df)   # this will stop and reset all variables

# stop("Stopping here")

# try:
#   raise 
# except Exception as e:
#  stop(f'Error here {e})
```

In [ ]:
# -------- CUSTOM TOOLS FOR DEBUGGING -------
def show(*args):
  # pandas display options
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  display(*args)
  pd.reset_option('display.max_rows')
  pd.reset_option('display.max_columns')

def stop(*args):
  """shows and stops, if first argument is True, it resets all variables"""
  show(*args)
  if isinstance(args[0], bool) and args[0]:
    print("****** Variables Reset *******")
    %reset -f # to skip prompt
  raise SystemExit

show("show() works")
for i in range(20):
  stop("stop() works. Yes, the exception is on purpose")
print("stop() doesn't work")

# Download Function
This is the near-final code for downloading and unpacking Yahoo, it will also unpack TDA and any other dictionary. It's a fancy json_normalize. Document any changes.

* Near-final beacuse it will have to include the deduplication code, which is unique to each database. This deduplication code is included in the Research area.
* This code has temporary lines demarked with # DEBUGGING, these are to find the source of the duplicated data.

In [ ]:
# --------  DOWNLOAD AND UNPACK ----------------
def download_unpack_yahoo():
  # symbols and chunks to download
  symbols = ['AAPL', 'FB', 'AMZN', 'TSLA', 'QQQ', 'BSX']
  chunk_size = len(symbols)  # no chunking

  # download stocks data in chunks and concatenate to large table
  df = pd.DataFrame()
  for i in range(0, len(symbols), chunk_size):
    symbol_chunk = symbols[i:i + chunk_size]
    tickers = yq.Ticker(symbols=symbol_chunk, asynchronous=True).all_modules
    chunk_df = pd.DataFrame(tickers).T
    df = pd.concat([df, chunk_df])

  # unpack dictionaries and concatenate to table
  # packages = ['summaryProfile', 'summaryDetail']  # by name
  packages = df.columns  # all

  # adding prefix to avoid duplicated label errors unpacking
  df = df.add_prefix("_")
  packages = ["_" + i for i in packages]

  df = pd.concat([pd.DataFrame(None, index=['package']), df])  # DEBUGGING
  for package in packages:
    on_hand = df.loc[df[package].notnull(), package]
    unpack_df = pd.json_normalize(on_hand).set_index(on_hand.index)
    unpack_df.loc['package'] = package[1:]  # DEBUGGING
    df.drop(package, axis=1, inplace=True)
    df = pd.concat([df, unpack_df], axis=1)

  # cleanup code
  # when done, insert the 'deal with duplicates' section here
  df.rename_axis('symbol', inplace=True)
  return df

yf = download_unpack_yahoo()
yf.T

# Research Area
Here we will find duplicated fields and we'll be able to download the list of fields in a CSV/GoogleSheet to be edited separately, then upload it, and then apply the corresponding dictionary to the DataFrame for testing. We'll need the dictionary. We may keep it in the dataset folder of the financial scanner.

* Using Google Colab Research, so you'll see many references to it. Disable it in the settings above ```google_colab = False```




In [ ]:
if google_colab:
    from google.colab import auth, drive, files
    from oauth2client.client import GoogleCredentials
    import gspread
    import io
    auth.authenticate_user()
    drive.mount("/content/drive")
    gc = gspread.authorize(GoogleCredentials.get_application_default())

Populate DataFrame with the latest download, or if Google Colab you can upload a CSV with the DataFrame.
This data changes every time, becuase we'll download with Async

In [ ]:
# download or use file for full DataFrame
if get_data:
  try:
    yf = download_unpack_yahoo()
  except:
    if google_colab:
      uploaded = files.upload()
      yf = pd.read_csv(io.BytesIO(uploaded[list(uploaded)[0]]))
      yf.drop(columns=['Unnamed: 0'], inplace=True)
    else:
      stop("Error getting data")
yf.head()

## Deduplication
Here is the code to find the duplicated fields. We need to either rename or delete these. This happens because two sources have the same field, i.e. SummaryProfile has quoteType and assetProfile has quoteType.
* List ```erase = ['maxAge', 'symbol']``` is a working example for YahooQuery
* The output of this section has to be a list or custom code that we can embed in the final code.



In [ ]:
# deal with duplicates
erase = ['maxAge', 'symbol']
try:
  yf.drop(erase, axis='columns', inplace=True, errors='ignore')
except AttributeError as e:
  stop(f"Dataframe may be empty. {e}")
df = yf.loc[:,yf.columns.duplicated(keep=False)].T.sort_index()
if not df.empty:
  print("Data has duplicate index, deal with this first.")
  stop(df.iloc[:, 0:5])

Here we can export the DataFrame columns for easier edit on CSV or Google Sheet.
* Advantage of Google Sheet is that we can share the sheet and we can both edit. Whether we use the feature or not, this time around.

In [ ]:
if export_df:
  # outgoing format
  df = pd.DataFrame([yf.columns.unique(), ''], index=['current', 'new']).T
  df = df[['new', 'current']]
  try:
    if to_csv:
      # save to csv
      df.to_csv(file+".csv", index=False, header=True)
      print(f'Saved CSV file {file}')
    elif google_colab:
      # save to Google Sheet
      file = file.split("/")[-1].split(".")[0]
      gc.create(file)
      ws = gc.open(file).get_worksheet(0)
      from gspread_dataframe import set_with_dataframe
      set_with_dataframe(ws, df)
      print(f'Saved Google Sheets file {file}')
    else:
      raise
  except FileNotFoundError as e:
      stop(f"Couldn't export DataFrame. {e}")

Here we can import the CSV or Google Sheet after being edited.

In [ ]:
if import_df:
  try:
    if import_csv:
        df = pd.read_csv(file+".csv")
    elif google_colab:
      # get from Google Drive
      file = file.split("/")[-1].split(".")[0]
      rows = gc.open(file).get_worksheet(0).get_all_values()
      df = pd.DataFrame.from_records(rows)
      # format similar to read_csv
      # df.set_index(0, drop=True, inplace=True)
      df = df.rename(columns=df.iloc[0]).drop(df.index[0])
    else:
      raise
  except FileNotFoundError as e:
    stop(f"Couldn't import the DataFrame. {e}")
  show(df)

# Final check
Here we'll see how the final DataFrame look like.
* Remember that here we rename the labels.
* Delete columns with NaN in 'current' definition

In [ ]:
if apply_to_df and import_df:
  # prep for dictionary
  df1 = df.loc[(df['new'].notnull()) & (df['new'] != '')]
  dt = df1.to_dict(orient='records')
  dt = {i['current']: i['new'] for i in dt}

  # renames the lables using the dictionary
  yf.rename(columns=dt, inplace=True)

  # deletes columns not in the dictionary, because were deemed unimportant
  drop_me = df.loc[~df['new'].str.strip().astype(bool)]['current'].to_list()
  yf.drop(columns=dropped_columns, inplace=True)

  print("\nChanges you made:\n"
        "-----------------")
  show(dt)
  print("\nFind your changes here:\n"
        "-----------------------")
  show(yf.T.iloc[:,1])